Set Catalog

In [0]:
spark.sql("USE CATALOG postnord")
print(spark.catalog.currentCatalog())

Creating and updating watermark table

In [0]:
from pyspark.sql.types import StructType, StructField, TimestampType
from pyspark.sql import Row

# Explicitly specify schema
schema = StructType([StructField("last_processed_dt", TimestampType(), True)])
initial_watermark = [Row(last_processed_dt=None)]
df_watermark = spark.createDataFrame(initial_watermark, schema)
df_watermark.write.format("delta").mode("overwrite").saveAsTable("control_watermark_table")

In [0]:
# Read watermark value
watermark_df = spark.read.table("control_watermark_table")
last_watermark = watermark_df.select("last_processed_dt").collect()[0][0]
#display(last_watermark)

load delta table

In [0]:
df_status = spark.table("silver_item_status")

In [0]:
# Filter new data
if last_watermark is None:
    new_data = df_status
else:
    new_data = df_status.filter(df_status.created_dt > last_watermark)

In [0]:
# after processing, update watermark
new_max = new_data.agg({"created_dt": "max"}).collect()[0][0]
# Overwrite the watermark table with the new value
spark.createDataFrame([Row(last_processed_dt=new_max)]).write.format("delta").mode("overwrite").saveAsTable("control_watermark_table")
#display(new_max)